In [2]:
import pandas as pd
import numpy as np
import sys
import os
import json
import datasets
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import *

aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']
# labels = pd.re{ad_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')
human_data = {}
for aspect in aspects:
    human_data[aspect] = []
    for conf in ['gold','silver', 'hard']:
        raw_data = datasets.load_dataset('boda/review_evaluation_human_annotation', aspect, split=conf).to_pandas()
        human_data[aspect].append(raw_data)

    human_data[aspect] = pd.concat(human_data[aspect])

    print(len(human_data))


1
2
3
4


In [7]:
final_data = {
}


for aspect in aspects:
    not_valid_points = 0
    final_data[aspect] = {
          'boda' : [],
    '6686ebe474531e4a1975636f' : [],
    '6740484e188a64793529ee77' : [],
    }
    aspect_data = human_data[aspect]

    for i, row in aspect_data.iterrows():
        # data = row[aspect].replace("'", "\"")
        data = row[aspect]
        # data = json.loads(data)
        labels = data['labels']
        annotators = data['annotators']
        for label, annotator in zip(labels, annotators):
            found = False
            if len(annotators) != 3:
                not_valid_points += 1
                continue
            ## search for the annotator in the unique id map

            final_data[aspect][annotator].append(label)
            # for key in final_data[aspect]:
            #     if annotator in annotators_unique_id_batch_id_map[key]:
            #         final_data[aspect][key].append(label)
            #         found = True
                    
            # if not found:
            #     print(f'annotator {annotator} not found in the unique id map')
               
    print(f'not valid points for {aspect} is {not_valid_points}')
        


not valid points for actionability is 0
not valid points for grounding_specificity is 0
not valid points for verifiability is 0
not valid points for helpfulness is 0


In [11]:
prinaple_name = {
    'boda' : 'boda',
    '6686ebe474531e4a1975636f' : 'Annotator_X',
    '6740484e188a64793529ee77' : 'Annotator_Y',
}

with open('human_annotators_agreemetn.txt', 'w') as f:

    for aspect in aspects:
        
        ## calculate the agreement between each pair of annotators
        annotators = ['boda', '6686ebe474531e4a1975636f', '6740484e188a64793529ee77']
        print(f'Aspect: {aspect}')
        print(f'Number of data points: {len(final_data[aspect][annotators[0]])}')

        aspect_average_results = { "f1": 0.0, "kappa": 0.0, "kappa_quadratic": 0.0,}
        if aspect == 'verifiability':
            aspect_average_results['f1_X'] = 0.0

        
        for i in range(len(annotators)):
            for j in range(i+1, len(annotators)):
                annotator1 = annotators[i]
                annotator2 = annotators[j]
                data1 = final_data[aspect][annotator1]
                data2 = final_data[aspect][annotator2]

                annotator1 = prinaple_name[annotator1]
                annotator2 = prinaple_name[annotator2]

                agreement = get_stats(data1, data2, aspect)

                for key in aspect_average_results:
                    aspect_average_results[key] += agreement[key]
        
        for key in aspect_average_results:
            aspect_average_results[key] /= 3
        f.write(f'Aspect: {aspect}\n')
        f.write(f'Average agreement between annotators\n')
        f.write(f'F1: {aspect_average_results["f1"]}\n')
        f.write(f'Kappa: {aspect_average_results["kappa"]}\n')
        f.write(f'Kappa Quadratic: {aspect_average_results["kappa_quadratic"]}\n')
        if aspect == 'verifiability':
            f.write(f'F1_X: {aspect_average_results["f1_X"]}\n')
        f.write('\n\n')


        




Aspect: actionability
Number of data points: 1055
Aspect: grounding_specificity
Number of data points: 1054
Aspect: verifiability
Number of data points: 1054
Aspect: helpfulness
Number of data points: 1054
